In [9]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

# Set random seed for reproducibility
np.random.seed(42)

In [ ]:
# Define categories and paths
categories = {'organic': 'biological', 'metal': 'metal', 'plastic': 'plastic'}
data_dir = './garbage-dataset/'  # Update if different
img_size = (224, 224)  # Standard size for CNNs like MobileNet

images = []
labels = []

for label, folder in categories.items():
    folder_path = os.path.join(data_dir, folder)
    if not os.path.exists(folder_path):
        raise ValueError(f"Folder not found: {folder_path}. Ensure dataset is downloaded.")
    
    for img_name in os.listdir(folder_path)[:150]:  # Limit to 150 per category for speed
        img_path = os.path.join(folder_path, img_name)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, img_size)
            images.append(img)
            labels.append(list(categories.keys()).index(label))  # 0: organic, 1: metal, 2: plastic

images = np.array(images)
labels = np.array(labels)

print(f"Loaded {len(images)} images across {len(categories)} categories.")

ValueError: Folder not found: ./garbage_classification/biological. Ensure dataset is downloaded.